## Análise de unigramas nos Tweets

In [1]:
#Descompactando os dados
import gzip
import shutil

with gzip.open('tweets_222_23abr2020.gz', 'rb') as entrada:
    with open('tweets_222_23abr2020.json', 'wb') as saida:
        shutil.copyfileobj(entrada, saida)

In [14]:
import pandas as pd
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation

In [19]:
#Ler dados
df = pd.read_json('tweets_222_23abr2020.json',convert_dates=False,lines=True,chunksize=200)



In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabrielapaixao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
stop_words = stopwords.words('portuguese') + list(punctuation)+ ['“','”','…','rt']
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False) 

def tokenize(text):
    tokenized_text = tweet_tokenizer.tokenize(text)
    filtered_words = [word for word in tokenized_text if word not in stop_words]
    return filtered_words

In [25]:
tweets = pd.Series(name = 'ID')

#testando com 5000 tweets
n=0
for i in df:
    n+=1
    if(n<5000):
        tweets = tweets.append(i['text'])
    else:
        break

tweets.head()

1104000    RT @marcofeliciano: 🚨URGENTE! Estudo comprova ...
1104001    RT @cienciausp: Para ajudara resolver o proble...
1104002    Vou mandar o significado de quarentena pro pes...
1104003          Quero que o corona vírus vá embora do mundo
1104004    RT @FabioTalhari: Mas o que significa isto? É ...
dtype: object

In [40]:
freq={} #número de ocorrências
num_tweets={} #número de tweets em que o unigrama ocorre

counts = [] #lista auxiliar de unigramas

for tweet in tweets.values:
    words = tokenize(tweet)
    counts.clear()
    for unigram in words:
        if unigram in freq:
            freq[unigram] += 1
            if unigram not in counts:
                num_tweets[unigram] += 1
                counts.append(unigram)
        else:
            freq[unigram] = 1
            num_tweets[unigram] = 1
            counts.append(unigram)


In [49]:
#apresentar em forma de tabela 
unigramas = pd.DataFrame([freq,num_tweets]).T.rename(columns={0:'Ocorrências',1:'Tweets'})

In [50]:
unigramas.sort_values('Ocorrências',ascending = False).head()

,Ocorrências,Tweets
quarentena,455328,444829
covid,116599,116092
19,101746,101327
pra,89646,84673
vai,79269,74313


In [51]:
unigramas.sort_values('Tweets',ascending = False).head()

,Ocorrências,Tweets
quarentena,455328,444829
covid,116599,116092
19,101746,101327
pra,89646,84673
corona,77648,76859


In [52]:
#salvar resultados em um csv
unigramas.to_csv('tweets-unigramas.csv',sep=';')